#### 1. Download Data and extrack 1 of 500 commercial vehicles' trajectories.

##### Download Data

In [0]:
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings

import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [0]:
import urllib.request
urllib.request.urlretrieve("https://figshare.com/ndownloader/files/33524447", "/tmp/myxxxx.csv")
dbutils.fs.mv("file:/tmp/myxxxx.csv", "dbfs:/EV_chargingMap/Xuancheng/FCD.csv")
display(dbutils.fs.ls("dbfs:/EV_chargingMap/Xuancheng/"))

path,name,size,modificationTime
dbfs:/EV_chargingMap/Xuancheng/FCD.csv,FCD.csv,3415556737,1740445971000
dbfs:/EV_chargingMap/Xuancheng/road_info.csv,road_info.csv,336428,1739896297000


In [0]:
urllib.request.urlretrieve("https://figshare.com/ndownloader/files/33524711", "/tmp/myxxxx.csv")
dbutils.fs.mv("file:/tmp/myxxxx.csv", "dbfs:/EV_chargingMap/Xuancheng/road_info.csv")
display(dbutils.fs.ls("dbfs:/EV_chargingMap/Xuancheng/"))

path,name,size,modificationTime
dbfs:/EV_chargingMap/Xuancheng/FCD.csv,FCD.csv,3415556737,1740445971000
dbfs:/EV_chargingMap/Xuancheng/road_info.csv,road_info.csv,336428,1740446140000


In [0]:
fcd_data_path = "dbfs:/EV_chargingMap/Xuancheng/FCD.csv"
road_data_path = "dbfs:/EV_chargingMap/Xuancheng/road_info.csv"

##### View Data

All Vehicle Data

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('vehicle analysis') \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df_fcd = spark.read.format("csv").option("header", "true").load(fcd_data_path)
display(df_fcd.limit(10))
df_fcd.createOrReplaceTempView("fcd_info")


cid,type,time,lon,lat,spd,turn,dis,roadid
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:23:50,118.76256894584729,30.94340203652462,3.127881054091683,Unknown,297.14870013870984,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:00,118.76252655644232,30.943123013046684,3.127881054091683,Unknown,265.869889597793,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:10,118.76248416739554,30.94284398952154,3.127881054091683,Unknown,234.59107905687623,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:20,118.76244177870673,30.942564965949177,3.127881054091683,Unknown,203.3122685159594,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:30,118.76239939037605,30.9422859423296,3.127881054091683,Unknown,172.03345797504258,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:40,118.76235700240338,30.942006918662816,3.127881054091683,Unknown,140.75464743412573,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:50,118.76231461478886,30.94172789494881,3.127881054091683,Unknown,109.47583689320889,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:25:00,118.76226876843316,30.941449473830712,3.127881054091683,Unknown,78.19702635229207,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:25:10,118.76218821730015,30.941177080781344,3.127881054091683,Unknown,46.91821581137524,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:25:20,118.7620626798798,30.94091817139982,3.127881054091683,Unknown,15.639405270458415,4724_4664


##### Extract vehicle with cid b62b0f461be5ee9702131b83ac8a1abb	and select only 2020-9-01 data

Data for cid b62b0f461be5ee9702131b83ac8a1abb at 2020-09-01

In [0]:
veh1_data = spark.sql("""
    SELECT * 
    FROM fcd_info 
    WHERE cid = 'b62b0f461be5ee9702131b83ac8a1abb' 
        AND time LIKE '2020-09-01%'""")
display(veh1_data.limit(10))

cid,type,time,lon,lat,spd,turn,dis,roadid
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:23:50,118.76256894584729,30.94340203652462,3.127881054091683,Unknown,297.14870013870984,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:00,118.76252655644232,30.943123013046684,3.127881054091683,Unknown,265.869889597793,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:10,118.76248416739554,30.94284398952154,3.127881054091683,Unknown,234.59107905687623,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:20,118.76244177870673,30.942564965949177,3.127881054091683,Unknown,203.3122685159594,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:30,118.76239939037605,30.9422859423296,3.127881054091683,Unknown,172.03345797504258,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:40,118.76235700240338,30.942006918662816,3.127881054091683,Unknown,140.75464743412573,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:24:50,118.76231461478886,30.94172789494881,3.127881054091683,Unknown,109.47583689320889,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:25:00,118.76226876843316,30.941449473830712,3.127881054091683,Unknown,78.19702635229207,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:25:10,118.76218821730015,30.941177080781344,3.127881054091683,Unknown,46.91821581137524,4724_4664
b62b0f461be5ee9702131b83ac8a1abb,2,2020-09-01 10:25:20,118.7620626798798,30.94091817139982,3.127881054091683,Unknown,15.639405270458415,4724_4664


Road Data

In [0]:
df_road = spark.read.format("csv").option("header", "true").load(road_data_path)
display(df_road.limit(10))
df_road.createOrReplaceTempView("road_info")

cid,nlane,turn,dnroad,geom,len
4393_4423,3,S#U#L,4423_4471#4423_4393#4423_4448,LINESTRING(118.723892282 30.9160036330001;118.724844511 30.9169356090001;118.725139689 30.9172075440001;118.72554912 30.9175847340001;118.72639217 30.9182988110001;118.727228216 30.918991981;118.728001335 30.9196325400001;118.728375699 30.9199501530001),612.7977763205269
4393_9052,0,L#S#U#R,9052_8367#9052_9051#9052_4393#9052_9135,LINESTRING(118.723892282 30.9160036330001;118.722024324 30.9140160600001;118.720700154 30.9125526920001;118.720136222 30.9119816210001;118.719768595 30.911646117;118.718447994 30.9105218220001;118.717744863 30.9098936440001;118.717195208 30.9094260800001;118.716467291 30.9088459870001),1066.4846532943966
4423_4471,4,R#S#U#L,4471_4440#4471_4493#4471_4423#4471_4496,LINESTRING(118.728375699 30.9199501530001;118.728747766 30.920265816;118.729801576 30.9211240270001;118.73076324 30.9219052030001;118.731603947 30.9226078240001;118.732520889 30.923362862;118.733360693 30.924013947;118.733407433 30.9240550690001),662.1959670657886
4438_4439,0,S#U,4439_4448#4439_4438,LINESTRING(118.718755032 30.9211743930001;118.720820615 30.920361588;118.722453919 30.9197707880001;118.722677839 30.919757067;118.722977489 30.9197996010001;118.723119303 30.91983708;118.72319199 30.9199152250001),462.0972608008144
4438_4474,2,R#S#U#L,4474_4448#4474_4582#4474_4438#4474_4502,LINESTRING(118.718755032 30.9211743930001;118.719776674 30.922589999;118.720795579 30.924005633;118.721023881 30.9243292560001),411.5472060925527
4439_4448,0,R#S#U#L,4448_4423#4448_4496#4448_4439#4448_4474,LINESTRING(118.72319199 30.9199152250001;118.723424185 30.92014691;118.724133865 30.9209544010001;118.72498767 30.9219444890001),283.04994985107373
4440_4497,3,R#U#S#L,4497_4415#4497_4440#4497_4526#4497_4536,LINESTRING(118.751056271 30.9209090950001;118.751191433 30.920961371;118.752246648 30.92134672;118.752827484 30.9215177930001;118.75335601 30.921665712;118.754085295 30.921819854;118.754786697 30.9219624400001;118.756260227 30.9221422540001;118.75744583 30.9222782470001;118.758355406 30.922453773;118.758758509 30.922552674;118.75941962 30.9227613960001;118.759952493 30.922956948;118.760632432 30.923248915;118.761156683 30.9235400170001;118.76200405 30.9240951310001;118.762639737 30.924614354;118.763323132 30.925282044;118.764629799 30.926687842),1500.5641761265365
4448_4423,2,L#R#U,4423_4471#4423_4393#4423_4448,LINESTRING(118.72498767 30.9219444890001;118.728375699 30.9199501530001),392.1221756222692
4448_4496,2,R#S#U#L,4496_4471#4496_4539#4496_4448#4496_4518,LINESTRING(118.72498767 30.9219444890001;118.725248578 30.922247036;118.726171057 30.9233131770001;118.726921554 30.9241716440001;118.728366735 30.9258606760001;118.729294252 30.9269730350001),693.016782274206
4464_4438,0,S#L#U,4438_4439#4438_4474#4438_4464,LINESTRING(118.709537063 30.923882081;118.710498958 30.92330087;118.714128082 30.921484804;118.715483571 30.9209085100001;118.715864184 30.92083546;118.716201188 30.9208775130001;118.716410172 30.920957055;118.716986881 30.9214039240001;118.717183735 30.92150152;118.71738391 30.921552546;118.717608155 30.921556724;118.717988449 30.921465777;118.718755032 30.9211743930001),1004.4727876181648


##### Generate Road map with given geom info from road data